In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Exploratory Data Analysis (EDA) 

## Load the data

In [ ]:
# Set paths
main_path = os.path.dirname(os.getcwd())
data_path = os.path.join(main_path, "data")
data_name = "dataset_SCL.csv"
df = pd.read_csv(os.path.join(data_path, data_name))

Conclusiones: 
- Se cargan los datos. 
- Existen problemas con la columna 1 y 6.

## Check for missing values

### Exploration

In [ ]:
df.isnull().sum()

Conclusiones: 
- Existe solo un valor nulo.

Acciones: 
- Ver que valor es el nulo

### Actions

Revisamos la fila con valor nulo.

In [ ]:
# create a boolean mask for missing values
mask = df.isna().any(axis=1)

# select the rows with missing values
rows_with_missing = df[mask]
rows_with_missing

Revisamos los posibles valores que puede tomar la columna Vlo-O cuando Vlo-I vale 200.

In [ ]:
df[df["Vlo-I"]  == "200"]["Vlo-O"].value_counts()

Dado que es un solo valor ("200"), se reemplaza.

In [ ]:
# FIll na value with 200
df.fillna("200", inplace=True)

## Summarize the data

### Shape: Explotarion

In [ ]:
df.shape

Conclusiones: 
- Se tienen 68206 y 18 columnas

### Column description: Exploration

In [ ]:
df.info()

Estas columnas represetan la siguiente información:

Section 1: Scheduled information of the flight
| Field | Description |
| --- | --- |
| Fecha-I | Scheduled date and time of the flight. |
| Vlo-I | Scheduled flight number. |
| Ori-I | Programmed origin city code. |
| Des-I | Programmed destination city code. |
| Emp-I | Scheduled flight airline code. |


Section 2: Real operated information of the flight
| Field | Description |
| --- | --- |
| Fecha-O | Date and time of flight operation. |
| Vlo-O | Flight operation number of the flight. |
| Ori-O | Operation origin city code. |
| Des-O | Operation destination city code. |
| Emp-O | Airline code of the operated flight. |

Section 3: Dates of flight operation
| Field | Description |
| --- | --- |
| DIA | Day of the month of flight operation. |
| MES | Number of the month of operation of the flight. |
| AÑO | Year of flight operation. |
| DIANOM | Day of the week of flight operation. |

Section 4: Type of the flight
| Field | Description |
| --- | --- |
| TIPOVUELO | Type of flight, I =International, N =National. |

Section 5: Name of codes used in previous sections
| Field | Description |
| --- | --- |
| OPERA | Name of the airline that operates. |
| SIGLAORI | Name city of origin. |
| SIGLADES | Destination city name. |

Conclusiones:
- Las variables Fecha-I y Fecha-O denerían tener formato de datetime.
- El resto de las variables son categorias (ej:  Un vuelo con un codigo 200, no tiene una unidad menos de algo que un vuelo con codigo 201. Esos numeros solo representan vuelos distintos (distintas categorias). )
- Las variables que son numericas se pueden dejar de esa forma por ahora (para efectos de hacer visualizaciones) pero es importante saber que en verdad son cateogoricas.

Ideas: 

- Las variables de la seccion 1 pueden ayudar a entender si existen rutas frecuentes, y si esas rutas tienen algun patron particular de retraso (variables necesarias: Vlo-I, Ori-I, Des-I). Además, pueden ayudar a ver si alguna areolinea tiene un comportamiento particular  de retraso (Emp-I).

    - Preguntas hasta ahora:
        - Para este análisis es mejor considerar lo -I o lo -O?
        - Cuanto difiere lo -I de lo -O?

- Las variables de la seccion 2 pueden ayudar a entender: que pasa cuando lo planeado -I difiere de lo ejecutado -O. 
    - Preguntas hasta ahora:
        - Para este análisis es mejor considerar lo -I o lo -O? 
         - Cuanto difiere lo -I de lo -O?


- Las variables de la seccion 3 pueden servir para analizar si ciertos patrones estacionarios, por ejemplo estaciones del año, dias de la semana (fin de semana) u otros, afectan a que un vuelo se retrase.
- La variable de la seccion 4 puede ayudar a entender si los vuelos nacionales o internaiconales tienen patrones distintos de atraso de vuelos.
- Como las variables de seccion 5 son solo el nombre de otros códigos, me interesa guardarlas como diccionarios, dado que probablemente no serán consideradas en el análisis de calculo de probabilidad. 
    - Observacion: revisé que esa sección corresponde con como operó el vuelo. No con lo scheduled.


Otras consideraciones: 
- Es interesante estudiar tambien la mezcla de algunar variables, por ejemplo: aerolinea y ruta, ruta y temporada, aerolinea y temporada, entra otras. 
- Es interesante estudiar la frecuencia de retuas y retrasos y otras características/retrasos.
Otras variables: Podría el tiempo en santiago influir de manera significante en que un vuelo se retrase?

Acciones:
- Se transforma las columnas Fecha-I y Fecha-O a datetime.

### Column description: Actions

In [ ]:
# convert dates to datetime format
df["Fecha-I"] = pd.to_datetime(df["Fecha-I"])
df["Fecha-O"] = pd.to_datetime(df["Fecha-O"])

### Preview of the data: Exploration

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

Conclusiones:
- Notamos que en la columna Vlo-O, existen valores que estan como float y como string. Esta columna debe ser str, dado que se trata de una categoria  (ej:  Un vuelo con un codigo 200, no tiene una unidad menos de algo que un vuelo con codigo 201. Esos numeros solo representan vuelos distintos).

Acciones:
- Se soluciona el tema anterior.

### Preview of the data: Action

In [ ]:
# Check if float values are not in type XX.0
for value in df["Vlo-I"].unique():
    if isinstance(value, float):
        if not int(value) == value:
            print(value)

In [ ]:
# Check if float values are not in type XX.0
for value in df["Vlo-O"].unique():
    if isinstance(value, float):
        if not int(value) == value:
            print(value)

In [ ]:
# Conver to int and then to str just the float values
def float_to_str(x):
    try:
        return str(int(float(x)))
    except:
        return x
    # if type(x) == type(2.) and not np.isnan(x):
    #     return str(int(x))
    # return x

df["Vlo-I"] = df["Vlo-I"].apply(lambda x: float_to_str(x))
df["Vlo-O"] = df["Vlo-O"].apply(lambda x: float_to_str(x))
df["Vlo-I"] = df["Vlo-I"].astype(str)
df["Vlo-O"] = df["Vlo-O"].astype(str)

## Pre  Analysis: Check differences between -I (scheduled) and -O (operated).
Check some variables before further exploration

### Question: From where the flights departue?

In [ ]:
departures_I, departures_O = df["Ori-I"].value_counts(), df["Ori-O"].value_counts()
departures_I, departures_O

### Answer: Los vuelos solo despegan desde SCEL, por lo que estas columnas no agregan ninguna información.

Acciones: 
- Se eliminan dichas columnas

In [ ]:
df.drop(["Ori-I", "Ori-O", "SIGLAORI"], inplace = True, axis = 1)

### Question: Donde van los vuelos? Is there any difference between Vlo-I and Vlo-O?

In [ ]:
departures_I = df["Des-I"].value_counts()
departures_I

In [ ]:
departures_O = df["Des-O"].value_counts()
departures_O

Cuantas veces estos valores son iguales?

In [ ]:
len(df[df["Des-I"] == df["Des-O"]])/(len(df))

Son iguales en el 99% de los casos.

Cuando no osn iguales?

In [ ]:
mask = df["Des-I"] != df["Des-O"]
df[mask].head(n=10)

Conclusiones:
- Existen variados destinos.
- Des-I es igual a Des-O en el 99% de los casos.

### Answer: Existen 63 destinos a los que los vuelos llegan. Se trabaja con Des-O.

In [ ]:
df.drop(["Des-I"], axis=1, inplace=True)

### Question: Is Vlo-I, Vlo-O usedful?

In [ ]:
# Check how many times are those variables equal
np.sum(df["Vlo-I"] == df["Vlo-O"])/len(df)

In [ ]:
# Check when they are not equal
mask = df["Vlo-I"] != df["Vlo-O"]
df[mask].head(n=10)

Conclusion:
- They are equal in a 99.8% of the cases. It seems that is not important which of those variables describe the airline that operates. Now those variables are just the code of the airline that operates the flight, the name is in the OPERA columns.
- As Vlo-O have the name of the airline that actually operate the flight, this variable is more useful.
- Ademas, si bien cambia el numero de vuelo, en general el destino y la empresa operadora se mantiene constate.

Acciones:
- Se elimina la variabre Vlo-I, dado que no es util.

In [ ]:
df.drop(["Vlo-I"], axis = 1, inplace = True)

Se analiza la variable Vlo-O

In [ ]:
# Group the data
grouped_df = df.groupby(['Vlo-O'])['Des-O'].unique()
multi_values = [grouped_df.index[i] for i in range(len(grouped_df)) if len(grouped_df[i]) > 1]
result = grouped_df.loc[multi_values]
result

In [ ]:
# Group the data
grouped_df = df.groupby(['Vlo-O'])['OPERA'].unique()
multi_values = [grouped_df.index[i] for i in range(len(grouped_df)) if len(grouped_df[i]) > 1]
result = grouped_df.loc[multi_values]
result

Conclusion:
- Uno número de vuelo se utiliza en varios casos para solo un destino, por lo que tendría alta correlacion con el destino de un vuelo, prefiero utilizar destino.
- Un número de vuelo puede pertenecer a más de una aerolinea. Dado que este es un problema de negocios, es interesante analizar como cada aerolinea se relaciona con una tasa de retraso, por lo que decide no utilizar esta columna.
- No se utilizara Vlo-O.

### Answer: None of those variables is useful.

In [ ]:
df.drop(["Vlo-O"], axis = 1, inplace = True)

### Question: How Emp-I, Emp-O and Opera are related? Which one(S) to keep?

Are they equal all the time?

In [ ]:
# Check how many times are those variables equal
np.sum(df["Emp-I"] == df["Emp-O"])/len(df)

No, just in 72% of the cases.

Each EMP-O has an unique OPERA?

In [ ]:
opera_Emp_dict = {opera: df.loc[df['OPERA'] == opera, 'Emp-O'].unique().tolist() for opera in df['OPERA'].unique()}
opera_Emp_dict

No, and actually all the Emp-O are in a unique claster of OPERA. Except for ['ARG', 'AUT'], que estan en 'Aerolineas Argentinas' y 'Austral'.

In [ ]:
mask = df["Emp-I"] != df["Emp-O"]
df[mask][["Emp-I","Emp-O", "OPERA"]].value_counts()

Conclusion:
- Notamos que en casi todos los casos, cueando EMP-I es distinto a EMP-O, el mismo OPERADOR se mantiene.

Excepciones:
- LAW no estaba en el diccionario, revisando en internet (ademas es claro), es parte de Latin American Wings.
- QFU y QFA son la unica diferencia, ya que no se tiene informacion de QFU. En internet no aparece QFU, pero dado la similitud de nombres, se asume que sigue siende Qantes Airways.
- Se decide que quien realmente opera los vuelos es OPERA, por ende se trabaja con esta columna.

### Answer: Se decide que quien realmente opera los vuelos es OPERA, por ende se trabaja con esta columna.

Acciones:
- Se elimina Emp-I y Emp-O

In [ ]:
# df.drop(["Emp-I", "Emp-O"], axis=1, inplace=True)

In [ ]:
df.columns

## Data Exploration

In [ ]:
df.columns

### Numeric variables distributions

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(14, 2))

ax[0].hist(df['DIA'], bins=31, color='red')
ax[0].set_xlabel('DIA')
ax[0].set_ylabel('Frequency')

ax[1].hist(df['MES'], bins=12, color='green')
ax[1].set_xlabel('MES')
ax[1].set_ylabel('Frequency')

ax[2].hist(df['AÑO'], bins=2, color='blue')
ax[2].set_xlabel('AÑO')
ax[2].set_ylabel('Frequency')

ax[3].hist(df['DIANOM'], bins=7, color='blue')
ax[3].set_xlabel('DIANOM')
ax[3].set_ylabel('Frequency')

plt.show()

Conclusiones:
- A primera vista no se ve nada raro con estas variables. Los días, meses, años y dianom estan en un rango adecuado. No se ven los datos centrados en algun valor en particular.
- Los datos son mayormente del 2017, a excepcion de dos vuelos que pasaron justo de la media noche del ultimo dia del 2017. (por retrasos)

### Categoric variables distributions

In [ ]:
df.columns

In [ ]:
def distribution_cat_variables(variable):
    # Group the data by airline and count the number of flights for each airline
    airline_counts = df[variable].value_counts()

    # Normalize the values to show proportions
    airline_proportions = airline_counts / airline_counts.sum()

    # Create a bar graph of airline market share
    plt.figure(figsize=(10, 2))
    airline_proportions.plot(kind='bar',cmap='viridis')
    plt.title(f'Number of flights by {variable}')
    plt.xlabel(f'{variable}')
    plt.ylabel('Proportion of Flights')
    plt.show()

In [ ]:
distribution_cat_variables('Des-O')

In [ ]:
distribution_cat_variables('TIPOVUELO')

In [ ]:
distribution_cat_variables('OPERA')

### Time variables

In [ ]:
# Group the data by month and flight type and count the number of flights for each group
month_type_counts = df.groupby(['MES', 'TIPOVUELO']).size().unstack()

# Create a line chart of number of flights by month and flight type
ax = month_type_counts.plot(kind='line', figsize=(5, 3))
ax.set_title('Number of Flights by Month and Flight Type')
ax.set_xlabel('Month')
ax.set_ylabel('Number of Flights')
plt.show()

Conclusiones:
- La estacionalidad de vuelos naciones e internacionales es la misma.

In [ ]:
# Resample the data to daily frequency and calculate the rolling mean with a window size of 7 days
daily_counts = df.set_index('Fecha-I').resample('D').size()
rolling_mean = daily_counts.rolling(window=30).mean()

# Plot the daily flight counts and the rolling mean
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(daily_counts.index, daily_counts.values, label='Daily counts')
ax.plot(rolling_mean.index, rolling_mean.values, label='7-day rolling mean')
ax.legend()

# Calculate and plot the trendline
x = np.arange(len(daily_counts))
coeffs = np.polyfit(x, daily_counts, 1)
trendline = np.poly1d(coeffs)
ax.plot(daily_counts.index, trendline(x), label='Trendline', linestyle='--')
ax.legend()

ax.set_title('Number of Flights per Day')
ax.set_xlabel('Date')
ax.set_ylabel('Number of Flights')
plt.show()

Conclusiones:
- Notamos que hay dos peak en la demanda de vuelos
- Un peak puede relacionarse con las vacaciones de invierno en Chile (finales de Julio - principios de Agosto)
- El otro peak puede relacionarse con las vacaciones de verano en Chile (finales de Noviembre - principios de Marzo)

# Generate additional columns

Se generan las siguientes columnas. Notemos que las siguientes columnas solo dependen de la información relativa a un vuelo, lo que es importante, ya que no generaran "Data leakage". Hay que tener eso en cuenta al generar variables que utilicen información historica.

## high_season
1 if Date-I is between Dec-15 and Mar-3, or Jul-15 and Jul-31, or Sep-11 and Sep-30, 0 otherwise.

In [ ]:
df["high_season"] = (
      ((df['Fecha-I'].dt.month == 12) & (df['Fecha-I'].dt.day >= 15)) | 
      (df['Fecha-I'].dt.month == 1) | 
      (df['Fecha-I'].dt.month == 2) | 
      ((df['Fecha-I'].dt.month == 3) & (df['Fecha-I'].dt.day <= 3)) | 
      ((df['Fecha-I'].dt.month == 7) & (df['Fecha-I'].dt.day >= 15) & (df['Fecha-I'].dt.day <= 31)) | 
      ((df['Fecha-I'].dt.month == 9) & (df['Fecha-I'].dt.day >= 11) & (df['Fecha-I'].dt.day <= 30))
   ).astype(int)

## min_diff
difference in minutes between Date-O and Date-I .

In [ ]:
df['min_diff'] = df.apply(lambda x: (x['Fecha-O'] - x['Fecha-I']).total_seconds() / 60, axis=1)

## delay_15
1 if min_diff > 15, 0 if not.

In [ ]:
df['delay_15'] = (df['min_diff'] > 15).astype(int)

## period_day
morning (between 5:00 and 11:59), afternoon (between 12:00 and 18:59) and night (between 19:00 and 4:59), based on Date-I .

In [ ]:
# Create the period_day variable
df.loc[(df['Fecha-I'].dt.hour >= 5) & (df['Fecha-I'].dt.hour < 12), 'period_day'] = 'morning'
df.loc[(df['Fecha-I'].dt.hour >= 12) & (df['Fecha-I'].dt.hour < 19), 'period_day'] = 'afternoon'
df.loc[((df['Fecha-I'].dt.hour >= 19) & (df['Fecha-I'].dt.hour <= 23)) | ((df['Fecha-I'].dt.hour >= 0) & (df['Fecha-I'].dt.hour < 5)), 'period_day'] = 'night'

## Guardar en .CSV

In [ ]:
df[["high_season", "min_diff", "delay_15", "period_day"]].to_csv(os.path.join(data_path, "synthetic_features.csv"))

# 3. Behavior of the delay rate across destination, airline, month of the year, day of the week, season, type of flight.

## Q : What variables would you expect to have the most influence in predicting delays?

### 3.1.1.  Delay rate across destination

In [ ]:
# Calculate delay rate by destination
delay_rate = df.groupby('Des-O')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 3))
sns.barplot(x=delay_rate.index, y=delay_rate.values, ax=ax, color='cornflowerblue')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Destination', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Destination', fontsize=18)
ax.tick_params(axis='x', labelrotation=70)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=5, padding=8)

plt.show()

In [ ]:
# Calculate delay rate and count of flights by destination
delay_rate = df.groupby('Des-O')['delay_15'].mean().sort_values(ascending=False)
flight_count = df['Des-O'].value_counts().sort_index()

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 5))

ax.bar(delay_rate.index, delay_rate.values, color='cornflowerblue', width=0.4, label='Delay Rate')
ax2 = ax.twinx()
ax2.bar(flight_count.index, flight_count.values, color='lightgreen', width=0.4, label='Count of Flights')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Destination', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax2.set_ylabel('Count of Flights', fontsize=14)
ax.set_title('Delay Rate and Count of Flights Across Destination', fontsize=18)
ax.tick_params(axis='x', labelrotation=70)

ax.legend(loc='upper right', fontsize=10, bbox_to_anchor=(1, 1.2))
ax2.legend(loc='upper left', fontsize=10, bbox_to_anchor=(0, 1.2))

# Add values on top of each bar
for i, v in enumerate(delay_rate.values):
    ax.text(i - 0.2, v + 0.01, f'{v:.2f}', fontsize=8)
for i, v in enumerate(flight_count.values):
    ax2.text(i + 0.2, v + 50, str(v), fontsize=8)

plt.show()

Conclusiones:
- Notamos que hay destinos con un delay rate mucho más alta que otros.
- Se ve que tambien hay destinos con delay rate 1 o 0. Es interesante notar que algunos de estos destinos tienen más de 100 vuelos. 
- Tambien hay algunos destinos con una tasa sobre el promedio, pero que tienen una baja cantidad de vuelos.


Es interesante incorporar esta variable al modelo. Se puede incorporar de dos maneras:
- Como una variable categorica (aunque sean muchas variables de este estilo)
- Como una variable númerica utilizando data historica (Delay_rate_per_airline)
- Seriea interesante ver si se puede combinar con la variable cantidad de vuelos

### 3.1.1.  Delay rate across airline

In [ ]:
# Calculate delay rate by airline
airline_delay_rate = df.groupby('Emp-O')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 3))
sns.barplot(x=airline_delay_rate.index, y=airline_delay_rate.values, ax=ax, color='cornflowerblue')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Airline', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Airlines', fontsize=18)
ax.tick_params(axis='x', labelrotation=45)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=8, padding=8)

plt.show()

In [ ]:
# Calculate delay rate and count of flights by employee responsible for delay
delay_rate = df.groupby('Emp-O')['delay_15'].mean().sort_values(ascending=False)
flight_count = df['Emp-O'].value_counts().sort_index()

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 5))

ax.bar(delay_rate.index, delay_rate.values, color='cornflowerblue', width=0.4, label='Delay Rate')
ax2 = ax.twinx()
ax2.bar(flight_count.index, flight_count.values, color='lightgreen', width=0.4, label='Count of Flights')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Employee Responsible for Delay', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax2.set_ylabel('Count of Flights', fontsize=14)
ax.set_title('Delay Rate and Count of Flights by Employee Responsible for Delay', fontsize=18)
ax.tick_params(axis='x', labelrotation=70)
ax.legend(loc='upper left', fontsize=12)
ax2.legend(loc='upper right', fontsize=12)

# Add values on top of each bar
for i, v in enumerate(delay_rate.values):
    ax.text(i - 0.2, v + 0.01, f'{v:.2f}', fontsize=8)
for i, v in enumerate(flight_count.values):
    ax2.text(i + 0.2, v + 50, str(v), fontsize=8)

plt.show()

In [ ]:
# Calculate delay rate and count of flights by employee responsible for delay
delay_rate = df.groupby('OPERA')['delay_15'].mean().sort_values(ascending=False)
flight_count = df['OPERA'].value_counts().sort_index()

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 5))

ax.bar(delay_rate.index, delay_rate.values, color='cornflowerblue', width=0.4, label='Delay Rate')
ax2 = ax.twinx()
ax2.bar(flight_count.index, flight_count.values, color='lightgreen', width=0.4, label='Count of Flights')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Employee Responsible for Delay', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax2.set_ylabel('Count of Flights', fontsize=14)
ax.set_title('Delay Rate and Count of Flights by Employee Responsible for Delay', fontsize=18)
ax.tick_params(axis='x', labelrotation=70)
ax.legend(loc='upper left', fontsize=12)
ax2.legend(loc='upper right', fontsize=12)

# Add values on top of each bar
for i, v in enumerate(delay_rate.values):
    ax.text(i - 0.2, v + 0.01, f'{v:.2f}', fontsize=8)
for i, v in enumerate(flight_count.values):
    ax2.text(i + 0.2, v + 50, str(v), fontsize=8)

plt.show()

### 3.1.1.  Delay rate across day of the week

In [ ]:
# Calculate delay rate by day of the week
delay_rate_dow = df.groupby('DIANOM')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(10, 6))
order = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
sns.barplot(x=delay_rate_dow.index, y=delay_rate_dow.values, ax=ax, color='cornflowerblue', order = order)

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Day of the Week', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Day of the Week', fontsize=18)
ax.tick_params(axis='x', labelrotation=45)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=10, padding=-11)

plt.show()


### 3.1.1.  Delay rate across month

In [ ]:
# Calculate delay rate by month
delay_rate_month = df.groupby('MES')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(14, 3))
sns.barplot(x=delay_rate_month.index, y=delay_rate_month.values, ax=ax, color='cornflowerblue')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Month', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Month', fontsize=18)
ax.tick_params(axis='x', labelrotation=45)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=10, padding=-11)

plt.show()

### 3.1.1.  Delay rate across season

In [ ]:
# Calculate delay rate by high_season
delay_rate_season = df.groupby('high_season')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(3, 3))
sns.barplot(x=delay_rate_season.index, y=delay_rate_season.values, ax=ax, color='cornflowerblue')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('High Season', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Season', fontsize=18)
ax.tick_params(axis='x', labelrotation=45)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=10, padding=-11)

plt.show()

### 3.1.1.  Delay rate across  type of flight

In [ ]:
# Calculate delay rate by high_season
delay_rate_type_flight = df.groupby('TIPOVUELO')['delay_15'].mean().sort_values(ascending=False)

# Calculate mean delay rate
mean_delay_rate = df['delay_15'].mean()

# Create barplot
fig, ax = plt.subplots(figsize=(3, 3))
sns.barplot(x=delay_rate_type_flight.index, y=delay_rate_type_flight.values, ax=ax, color='cornflowerblue')

# Add mean line
ax.axhline(mean_delay_rate, color='crimson', linestyle='--', linewidth=2, label='Mean Delay Rate')

# Set labels and titles
ax.set_xlabel('Type of Flight', fontsize=14)
ax.set_ylabel('Delay Rate', fontsize=14)
ax.set_title('Delay Rate Across Type of Flight', fontsize=18)
ax.tick_params(axis='x', labelrotation=45)
ax.legend(fontsize=12)

# Add values on top of each bar
for i in ax.containers:
    ax.bar_label(i, label_type='edge', labels=[f"{x:.2f}" for x in i.datavalues], fontsize=10, padding=-11)


plt.show()

# 4